In [23]:
import numpy as np
from shape_model.gaussian_ellipsoid import gaussian_ellipsoid_shape_model
from analytical_scattering_theories.homogeneous_sphere import mie_compute_q_and_s
from bl_dda.scatterer import Target, IncidentField, DiscreteDipoles



In [24]:
# Define the ellipsoid parameters a >= b >= c
#### length_unit= um ###
r_v_base= 0.5 # allowed range = [0.1, 0.5], volume equivalent radius of base ellipsoid
bc_ratio= 6 # allowed range = [1.0, 7.0], b/c ratio of base ellipsoid
ab_ratio= 1 # allowed range = [1.0, 2.0], a/b ratio of base ellipsoid
beta= 0.0 # standard deviation of GE surface deformation, allowed range= [0, 0.3]

In [25]:
gre_shape_model= gaussian_ellipsoid_shape_model(r_v_base, bc_ratio, ab_ratio, beta)

In [26]:
rng= np.random.default_rng(1234)
r_points_on_GRE_surf, xyz_meshes_GRE_surf = gre_shape_model.compute_r_points_on_GRE(rng)
lattice_domain, lattice_n, lattice_grid_points= gre_shape_model.create_cuboid_lattice_that_encloses_GRE_shape(r_points_on_GRE_surf)
dist_from_GRE= gre_shape_model.find_nearest_distance_from_the_GRE_surf(lattice_grid_points, r_points_on_GRE_surf)
lattice_grid_points_is_in_target= gre_shape_model.extract_lattice_address_in_GRE_volume(gre_shape_model.lattice_lf, gre_shape_model.distance_factor, lattice_n, dist_from_GRE)

In [27]:
wl_0= 0.834
m_m=1.329
#m_m=1.0

'''
#### Refractive index data of minerals from mindat.org (@ mid-visible wavelength)
#Quartz Uniaxial, positive
m_p_x= 1.544+0j
m_p_y= 1.544+0j
m_p_z= 1.553+0j

#Mica(Muscovite) Biaxial
# m_p_x= 1.564+0j
# m_p_y= 1.599+0j
# m_p_z= 1.603+0j

#Kaolinite Biaxial
m_p_x= 1.558+0j
m_p_y= 1.564+0j
m_p_z= 1.565+0j

#Calcite Uniaxial, negative
m_p_x= 1.658+0j
m_p_y= 1.658+0j
m_p_z= 1.486+0j

#Aragonite Biaxial
m_p_x= 1.5295+0j
m_p_y= 1.681+0j
m_p_z= 1.6855+0j
'''

'''
#Bacteria
m_p_x= 1.36+0j
m_p_y= 1.36+0j
m_p_z= 1.36+0j

#Organics
m_p_x= 1.5+0j
m_p_y= 1.5+0j
m_p_z= 1.5+0j
'''

m_p_x= 1.5+0j
m_p_y= 1.5+0j
m_p_z= 1.5+0j


m_p_xyz= np.array([m_p_x, m_p_y, m_p_z], dtype=np.complex64)

m_p_avg= (m_p_x+m_p_y+m_p_z)/3
#m_p_xyz= np.array([m_p_avg, m_p_avg, m_p_avg], dtype=np.complex64)
m_p_avg

(1.5+0j)

In [28]:

target= Target(gre_shape_model.name, lattice_n, gre_shape_model.lattice_lf, lattice_grid_points, lattice_grid_points_is_in_target, m_p_xyz)
lattice_n
# np.prod(lattice_n)= 269705
lattice_n, np.prod(lattice_n)

(array([57, 57, 12], dtype=int32), np.int64(38988))

In [29]:
rng= np.random.default_rng()
number_of_orientations= 10
euler_alpha_deg= rng.uniform(0,360, number_of_orientations).reshape(number_of_orientations,1)
euler_beta_deg= rng.uniform(0,180, number_of_orientations).reshape(number_of_orientations,1)
euler_gamma_deg= rng.uniform(0,360, number_of_orientations).reshape(number_of_orientations,1)

# euler_alpha_deg= np.array([0,45+180], dtype=np.float32).reshape(number_of_orientations,1)
# euler_beta_deg= np.array([0,30], dtype=np.float32).reshape(number_of_orientations,1)
# euler_gamma_deg= np.array([0,0], dtype=np.float32).reshape(number_of_orientations,1)

euler_angles_radian= np.radians(np.hstack((euler_alpha_deg,euler_beta_deg,euler_gamma_deg)))

incident_field= IncidentField(wl_0, m_m, euler_angles_radian)

np.hstack((euler_alpha_deg,euler_beta_deg,euler_gamma_deg))

array([[228.97005949, 113.04002977, 296.38163973],
       [166.43628225, 118.59666333, 250.4219088 ],
       [227.3453358 ,  87.93831159, 195.0463564 ],
       [ 99.77160177, 136.77823906, 115.79095921],
       [101.60978897, 140.49270221, 228.75167353],
       [319.8891886 ,  74.23480293, 305.56157583],
       [160.40599585,  95.85483488, 278.73230881],
       [199.01805315, 143.32925033,  98.86763986],
       [ 69.10371934, 169.17899246,  46.02989842],
       [ 86.88105652,  36.19960757,  91.52788054]])

In [30]:
discrete_dipoles= DiscreteDipoles(target, incident_field)

Number of dipoles per wavelength in the particle volume: dpl= 16.7451695998093


In [31]:
discrete_dipoles.set_interaction_matrix()

In [32]:
discrete_dipoles.solve_matrix_equation()

#ray version nprocs=4, 42.7s 
#ray version nprocs=8, 34s 
#nonray version , 1m36.3s


Serial computing of block-Matrix solver (single CPU core)
Starting block-BiCGStab iterative solver...
iter= 0, err= 0.0684
iter= 1, err= 0.0103
iter= 2, err= 0.0012
block-BiCGStab converged! (iter_fin=2, err_fin=0.0012, solver time=56.8s)


In [33]:
r_p= discrete_dipoles.ve_radius
r_p= r_v_base
m_p_avg= (m_p_x+m_p_y+m_p_z)/3
Q_sca_mie, Q_abs_mie, Q_ext_mie, S_fw_PCAS_mie, S_bk_OCBS_mie = mie_compute_q_and_s(wl_0,m_m,r_p,m_p_avg,nang=3)
Q_ext_mie, S_fw_PCAS_mie, S_bk_OCBS_mie

(np.float64(0.7847957152487692),
 np.complex64(1.0088559+0.4911062j),
 np.complex64(-0.018235039-0.032022126j))

In [34]:
C_abs= discrete_dipoles.compute_C_abs()
Q_abs= C_abs/(np.pi*discrete_dipoles.ve_radius**2)
Q_abs/Q_abs_mie, Q_abs, Q_abs_mie

(array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 np.float64(1.1102230246251565e-16))

In [35]:
C_ext= discrete_dipoles.compute_C_ext()
Q_ext= C_ext/(np.pi*discrete_dipoles.ve_radius**2)
Q_ext/Q_ext_mie

array([1.02758306, 0.90369765, 1.19359696, 0.47978198, 0.44670224,
       1.13177874, 1.18825909, 0.43377819, 0.37370039, 0.43206264])

In [36]:
S_fw_PCAS_theta_P, S_fw_PCAS_phi_P= discrete_dipoles.compute_PCAS_observable_S_fw()
S_fw_PCAS_theta_P, S_fw_PCAS_mie

(array([1.22176797+0.54396457j, 1.04687613+0.39393117j,
        1.3840971 +0.58113186j, 1.07164379+0.2597474j ,
        1.1020267 +0.23915296j, 1.08568634+0.60816632j,
        1.15669808+0.57531582j, 1.13653061+0.22456291j,
        1.16593702+0.19556366j, 1.12695165+0.23525481j]),
 np.complex64(1.0088559+0.4911062j))

In [37]:
S_fw_PCAS_phi_P, S_fw_PCAS_mie

(array([1.02526067+0.53547437j, 1.10649406+0.55537062j,
        1.16546818+0.6726987j , 1.11441203+0.24424627j,
        1.12738881+0.23009164j, 1.30333215+0.58072643j,
        1.37131925+0.6729075j , 1.11956612+0.23110543j,
        1.16449714+0.19699512j, 1.13306893+0.21861141j]),
 np.complex64(1.0088559+0.4911062j))

In [38]:
S_bk_OCBS_P= discrete_dipoles.compute_OCBS_observable_S_bk()

In [39]:
S_bk_OCBS_P, S_bk_OCBS_mie


(array([-0.0028817 -0.01330482j,  0.01129138-0.00382484j,
        -0.01090412-0.02520381j, -0.02528681-0.00289212j,
        -0.0350386 -0.00702746j, -0.0148369 -0.01699949j,
        -0.00996925-0.02756896j, -0.03123999-0.00533929j,
        -0.10358155-0.02565805j, -0.02660257-0.00399615j]),
 np.complex64(-0.018235039-0.032022126j))